<a href="https://colab.research.google.com/github/ellyasiml/CNN-LSTM-sentiment-analysis/blob/main/ta_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install symspellpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import re
import string
import ast
import gensim
from symspellpy import SymSpell, Verbosity

import nltk
nltk.download('all')
from nltk import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

<h1>Import Data

In [ ]:
df=pd.read_csv("drive/My Drive/dataset/dataset TA/data3.csv", encoding="utf-8")
df.head()

,date,id,username,location,tweet
0,2023-04-24 23:57:35+00:00,1.650000e+18,gunoto🇲🇨,NaN,b'RT @saiful_mujani: kl dari sisi pemilih calo...
1,2023-04-24 23:50:02+00:00,1.650000e+18,Republika.co.id,"Jakarta, Indonesia",b'PDIP tak membuka peluang Ganjar untuk menjad...
2,2023-04-24 23:30:53+00:00,1.650000e+18,🫡sadjoker,Kuala Lumpur City,b'RT @Mira_SasMiita: Setelah PDIP mendeklarasi...
3,2023-04-24 22:42:31+00:00,1.650000e+18,Haryoko,Indonesia,b'RT @KhoirilAnwar_: Gerindra: Prabowo Calon P...
4,2023-04-24 22:31:48+00:00,1.650000e+18,@asban_0577,NaN,b'RT @DPP_PPP: Plt Ketua Umum Partai Persatuan...


In [ ]:
# Read the CSV file using pandas
df = pd.read_csv("drive/My Drive/dataset/dataset TA/data3.csv", encoding='utf-8')

# Decode all elements in the dataframe using the UTF-8 encoding
df = df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

df

,date,id,username,location,tweet
0,2023-04-24 23:57:35+00:00,1.650000e+18,gunoto🇲🇨,NaN,b'RT @saiful_mujani: kl dari sisi pemilih calo...
1,2023-04-24 23:50:02+00:00,1.650000e+18,Republika.co.id,"Jakarta, Indonesia",b'PDIP tak membuka peluang Ganjar untuk menjad...
2,2023-04-24 23:30:53+00:00,1.650000e+18,🫡sadjoker,Kuala Lumpur City,b'RT @Mira_SasMiita: Setelah PDIP mendeklarasi...
3,2023-04-24 22:42:31+00:00,1.650000e+18,Haryoko,Indonesia,b'RT @KhoirilAnwar_: Gerindra: Prabowo Calon P...
4,2023-04-24 22:31:48+00:00,1.650000e+18,@asban_0577,NaN,b'RT @DPP_PPP: Plt Ketua Umum Partai Persatuan...
...,...,...,...,...,...
22204,2023-04-17 07:09:35+00:00,1.650000e+18,CharlyJo,NaN,RT @akbar84_: Nama Moeldoko kembali menduduki ...
22205,2023-04-17 07:07:28+00:00,1.650000e+18,Abdul Kholil,San Andreas,"""Kalau ke depan elektabilitas Erick Thohir nai..."
22206,2023-04-17 07:07:18+00:00,1.650000e+18,Abdul Kholil,San Andreas,Menteri BUMN Erick Thohir memiliki peluang jad...
22207,2023-04-17 07:01:59+00:00,1.650000e+18,Abdul Kholil,San Andreas,"Untuk calon wakil presiden harapan rakyat, Moe..."


<h1>Preprocessing

<h2>Data cleaning

In [ ]:
def correct_spelling(text):
    # Load the symspell dictionary
    sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
    dictionary_path = "drive/My Drive/dataset/dataset TA/wiki-id-formatted-1000.txt"
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

    # Split the text into words
    words = text.split()

    # Iterate over each word and correct its spelling
    corrected_words = []
    for word in words:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            corrected_words.append(suggestions[0].term)
        else:
            corrected_words.append(word)

    # Join the corrected words and return the corrected text
    return " ".join(corrected_words)

In [ ]:
def clean_tweet(tweet_text):    
    clean_text=tweet_text
    #remove retweet text (RT @...)
    clean_text=re.sub(r"b'","",clean_text)
    clean_text=re.sub(r"RT\s@\w+:","",clean_text)
    clean_text=re.sub(r"&amp;"," ",clean_text)
    #clean_text=re.sub(r"@aniesbaswedan","aniesbaswedan",clean_text)
    #clean_text=re.sub(r"@ganjarpranowo","ganjarpranowo",clean_text)
    #clean_text=re.sub(r"@puanmaharani_ri","puanmaharani_ri",clean_text)
    #clean_text=re.sub(r"@prabowo","prabowo",clean_text)
    #clean_text=re.sub(r"@ridwankamil","ridwankamil",clean_text)
    clean_text=re.sub(r"@\w+","",clean_text)
    #remove hashtags
    clean_text=re.sub(r"#\w+","",clean_text)
    #remove URLs
    url_regex=r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"
    clean_text=re.sub(url_regex,"",clean_text)
    #lower case
    clean_text=clean_text.lower()

    #tokenize each word
    word_list=word_tokenize(clean_text)
    clean_list=[]
    stop_words=stopwords.words("indonesian")
    for word in word_list:
        if word.isalpha() and word not in stop_words:
            clean_list.append(word)
            
    return " ".join(clean_list)

In [ ]:
print("Before: ", df.iloc[185]["tweet"])
df["tweet"]=df["tweet"].apply(clean_tweet)
#df["tweet"]=df["tweet"].apply(correct_spelling)
print("After: ", df.iloc[185]["tweet"])

Before:  b'@robert_yulian1 @gibran_tweet Jika gugatan PSI terhadap batas usia minimal capres/cawapres di pasal 169 \xe2\x80\x9cPersyaratan mnjd calon Presiden&amp;calon wakil presiden. Adlh berusia plg rendah 40 (empat puluh) tahun.\xe2\x80\x9ddikabulkan MK,MK bts usia minimum capres&amp;cawapres adh 35thn. \n\nDuet prabowo&amp;gibran sgt mungkin terjadi.'


KeyboardInterrupt: ignored

In [ ]:
df.to_csv("data_clean.csv", sep=",")